In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/All Projects/Learning Assistant/' -r '/content/'

In [ ]:
!pip install selfcheckgpt

In [ ]:
!pip install --upgrade flask-ngrok
!pip install pyngrok==4.1.1
!ngrok authtoken '2aAh0gs0N904HTHZSkIHm7tUbji_uSF52Avhxfw2axULYP27'

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=06078015b629c0b069d2c4ac584b1c5bb15d17ac203255a698f9afcfa7383a92
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import torch
from selfcheckgpt.modeling_mqag import MQAG

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

mqag_model = MQAG(
    g1_model_type='race', # race (more abstractive), squad (more extractive)
    device=device
)

cuda
MQAG (race) initialized to cuda


In [ ]:
%cd '/content/Learning Assistant/Module 3/'

/content/Learning Assistant/Module 3


In [ ]:
import numpy as np

def generate_questions_answers(context):
  questions = mqag_model.generate(context=context, do_sample=True, num_questions=10)
  final_set = []
  for i, question_item in enumerate(questions):
      question = question_item['question']
      options = question_item['options']

      single_question = [{'question': question, 'options': options}]
      probs = mqag_model.answer(questions=single_question, context=context)
      correct_answer = options[np.argmax(probs)]

      final_set.append({'question': question, 'options': options, 'answer': correct_answer})

  return final_set

In [ ]:
from flask import Flask, render_template, request, redirect, url_for , jsonify
import json
# from flask_ngrok import run_with_ngrok
from pyngrok import ngrok



app = Flask(__name__)
# run_with_ngrok(app)


global answers
global quiz_data_v2
global questionText
questionText = ""


with open("details.json", "r") as jf:
    data = json.load(jf)
    users = data["login"]


@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Retrieve form data
        email = request.form.get('login-email')
        password = request.form.get('login-password')

        if email in users.keys() and users[email]==password:
            return redirect(url_for('generate_qna'))

    return render_template('auth.html')


@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        # Retrieve form data
        email = request.form.get('signup-email')
        password = request.form.get('signup-password')
        password2 = request.form.get('confirm-signup-password')

        # Perform validation or other actions with the form data
        with open("details.json", "w") as jfk:
            users[email] = password
            json.dump(users, jfk)

        # Print form data (for demonstration purposes)
        print(f'Email: {email}, Password: {password} ,  password2 : { password2}')

        # Add your logic for user registration or other actions
        return redirect(url_for('login'))

    return render_template('auth.html')




@app.route('/create_post' , methods=['GET', 'POST'] )
def create_post():
    # Additional logic for generating Q&A or render a template
     if request.method == 'POST':
            # Retrieve form data
        new_post = request.form.get('post-input')


        # Perform validation or other actions with the form data

        # Print form data (for demonstration purposes)
        print(f'new_post: {new_post}')
        return redirect(url_for('forum'))




@app.route('/generate_qna' , methods=['GET', 'POST'])
def generate_qna():
    # Additional logic for generating Q&A or render a template
    question_data = [
    {"subject": "Math", "num_questions": 10},
    {"subject": "Physics", "num_questions": 10},
    # Add more data as needed
]

    return render_template('generate_qna.html' , questions=question_data)


@app.route('/answer_evaluation')
def answer_evaluation():

    subj_data = data["scores"]
    subjects_data = [
        {"subject": list(subj_data.keys())[0], "score": subj_data[list(subj_data.keys())[0]]},
        {"subject": list(subj_data.keys())[1], "score": subj_data[list(subj_data.keys())[1]]},
        {"subject": list(subj_data.keys())[2], "score": subj_data[list(subj_data.keys())[2]]},
    ]

    return render_template('answer_evaluation.html', subjects_data=subjects_data)
    # Additional logic for generating Q&A or render a template



@app.route('/forum')
def forum():
    # Additional logic for generating Q&A or render a template
    post_data = [
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        },
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        },
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        }
    ]
    return render_template('forum.html' ,post_data=post_data )


@app.route('/quiz' , methods=['GET', 'POST'])
def quiz():
    global answers
    global quiz_data_v2
    global questionText
    quiz_data = [

    {
        "question": "For Fiscal year 2021, what was your total GhG Carbon Emission for all scopes?",
        "options": [
            "Scope 1 - Determine environmental impact levels.",
            "Scope 2 - Reduce carbon footprints.",
            "Scope 3 - Enhance environmental impacts on a larger scale.",
            "I do not know the answer to this question.",
        ],
    },
    {
        "question": "Another question?",
        "options": [
            "Option 1",
            "Option 2",
            "Option 3",
            "I do not know the answer to this question.",
        ],
    },


    # Add more questions and answers as needed
    ]
    if request.method == 'POST':
        print("post method ************")
        questionText = request.form.get('questionText')
        print(f"Launching quiz for subject: {questionText}")
        quiz_data = generate_questions_answers(questionText.replace('\n', ' '))

        quiz_data_v2 = []
        answers = []
        for item in quiz_data:
            # d = ast.literal_eval(item)
            js_data = {
                "question": item['question'],
                "answers": item['options']
            }
            quiz_data_v2.append(js_data)
            answers.append(item['answer'])
        print(quiz_data_v2)
        print("Answers:", answers)

        # return redirect(url_for('quiz'))
        render_template('quiz.html' , quiz_data=quiz_data_v2)

    return render_template('quiz.html', quiz_data=quiz_data_v2)


@app.route('/quiz/response', methods=['POST'])
def quiz_response():
    global quiz_data_v2
    global answers

    if request.method == 'POST':
        # Retrieve the selected answers from the form
        selected_answers = []

        for question in quiz_data_v2:
            selected_answers.append(request.form.get(f'selected-answer-{question["question"]}'))


        # You can now process the selected answers as needed
        # For example, you might want to store them in a database or perform some analysis

        # For now, let's just print the selected answers
        print("Selected Answers:", selected_answers)
        score = 0
        quiz_data_ans = []

        for i in range(len(selected_answers)):
          quiz_data_ans.append({'question': quiz_data_v2[i]['question'], 'answer':answers[i]})
          if selected_answers[i]==answers[i]:
            score += 10


        result = "Fail" if score<50 else "Pass"


        # quiz_data_ans = [

        #         {
        #             "question": "For Fiscal year 2021, what was your total GhG Carbon Emission for all scopes?",
        #         "answer": "Scope 3 - Enhance environmental impacts on a larger scale."
        #     },
        #     {
        #         "question": "Another question?",
        #         "answer": "Option 3"
        #     },

        # ]

        # You can redirect to a different page or render a new template
        return render_template('QuizResult.html', result=result , score=score , quiz_data_ans=quiz_data_ans )
        # You might want to render a response or redirect to another page
    return render_template('quiz.html', quiz_data=quiz_data_v2)


@app.route('/logout')
def logout():
    # Additional logic for generating Q&A or render a template
    return redirect(url_for('login'))


ngrok_tunnel = ngrok.connect(5000)
print(ngrok_tunnel)
# print('Running at:', ngrok_tunnel.public_url)

if __name__ == '__main__':
    app.run(port=5000)


http://5cd1-34-125-159-25.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:17] "GET /static/js/auth.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:17] "GET /static/css/auth.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:18] "GET /static/Assets/Container.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:20] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:25] "POST / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:25] "GET /generate_qna HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:13:26] "GET /static/css/generate_qna.css HTTP/1.1" 200 -


post method ************
Launching quiz for subject: Plants are the eukaryotes that form the kingdom Plantae; they are predominantly photosynthetic. This means that they obtain their energy from sunlight, using chloroplasts derived from endosymbiosis with cyanobacteria to produce sugars from carbon dioxide and water, using the green pigment chlorophyll. Exceptions are parasitic plants that have lost the genes for chlorophyll and photosynthesis, and obtain their energy from other plants or fungi.

Historically, as in Aristotle's biology, the plant kingdom encompassed all living things that were not animals, and included algae and fungi. Definitions have narrowed since then; current definitions exclude the fungi and some of the algae. By the definition used in this article, plants form the clade Viridiplantae (green plants), which consists of the green algae and the embryophytes or land plants (hornworts, liverworts, mosses, lycophytes, ferns, conifers and other gymnosperms, and flowerin

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:18:46] "POST /quiz HTTP/1.1" 200 -


[{'question': 'Which of the following is NOT true according to the passage?', 'answers': ['Plants are all green algae and some of the algae.', 'Plants obtain their energy from sunlight.', 'Plants are predominantly photosynthetic.', 'Plants are all the eukaryotes that form the kingdom Plantae.']}, {'question': 'According to The text, plants _.', 'answers': ['cannot survive without sunlight', 'need the energy of others', 'need the energy of fungi', 'lack energy from fungi']}, {'question': 'Historically, the definition of the plant Kingdom did NOT include _.', 'answers': ['the glaucophytes', 'organelles', 'some fungi', 'embryophytes']}, {'question': 'According to the text, parasitic plants _.', 'answers': ['get energy from other plants or fungi', 'are photosynthetic', 'have many members', 'have no chlorophyll and photosynthesis']}, {'question': 'According to aristotle\'s biology, the word "plants" now most often mean _.', 'answers': ['plants that produce energy from sunlight', 'plants tha

INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:19:10] "POST /quiz HTTP/1.1" 200 -


[{'question': 'What can we infer from the definition used in the text?', 'answers': ['The Viridiplantae makes up the land plants.', 'Arcaplastida is a kind of algae.', 'The clade Archaeplastida is a class of plants.', 'The Viridiplantae can be distinguished from the glaucioloopsis.']}, {'question': 'How does the writer give a complete overview of the division of the plant kingdom?', 'answers': ['By providing a context.', 'By comparing plant kingdom with animal kingdom.', 'By giving an example of each unit.', 'By giving different information.']}, {'question': 'If you have some algae and a plant kingdom for a living, what kind of plant will you have then?', 'answers': ['The Viridiplantae', 'the plants outside the kingdom', 'The clade Archaeplastida', 'The Gramnaeae']}, {'question': 'What can we know from the passage?', 'answers': ['Now all plants belong to the clade Viridiplantae', 'Aristotle thought that plants were not animals', 'Plants formed two kingdoms in the past', 'Now all plants

INFO:werkzeug:127.0.0.1 - - [08/Jan/2024 10:19:11] "GET /static/css/quiz.css HTTP/1.1" 200 -
